In [ ]:
import os
for dirname, _, filenames in os.walk('drive/MyDrive/data/kvasir_small'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

drive/MyDrive/data/kvasir_small/val/dyed-lifted-polyps/23b2f181-4cd7-4a89-b157-16a6d275c0e1.jpg
drive/MyDrive/data/kvasir_small/val/dyed-lifted-polyps/98b17e7a-7e56-4e22-bf2d-3a4586366172.jpg
drive/MyDrive/data/kvasir_small/val/dyed-lifted-polyps/f8f88db2-257a-446d-a935-a457710d9091.jpg
drive/MyDrive/data/kvasir_small/val/dyed-lifted-polyps/eede44ce-1540-4851-ae95-ea36954e2ad5.jpg
drive/MyDrive/data/kvasir_small/val/dyed-lifted-polyps/301718e9-bbdf-4cdd-a665-908c36ae31ea.jpg
drive/MyDrive/data/kvasir_small/val/dyed-lifted-polyps/a6d1bb4f-e9fb-46b7-bdbf-babaf0b06581.jpg
drive/MyDrive/data/kvasir_small/val/dyed-lifted-polyps/56de76fd-43e8-44d4-ac96-3a8bbaf2a7dd.jpg
drive/MyDrive/data/kvasir_small/val/dyed-lifted-polyps/5ff2573f-c5e7-4d59-a01a-33f00152551f.jpg
drive/MyDrive/data/kvasir_small/val/dyed-lifted-polyps/140670b4-221e-4149-b748-f29940067147.jpg
drive/MyDrive/data/kvasir_small/val/dyed-lifted-polyps/b43a4c4c-7b97-4af9-a31c-316fb28b43f3.jpg
drive/MyDrive/data/kvasir_small/val/dyed

In [ ]:
TRAIN_DIR = 'drive/MyDrive/data/kvasir_small/train'
TEST_DIR = 'drive/MyDrive/data/kvasir_small/test'
VAL_DIR = 'drive/MyDrive/data/kvasir_small/val'

In [ ]:
import keras
import tensorflow

In [ ]:
from tensorflow.keras.layers import Flatten,Dense,Activation

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
from sklearn.metrics import confusion_matrix,accuracy_score,recall_score,precision_score,f1_score

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                    shear_range=0.2,
                                    zoom_range=0.2,
                                    horizontal_flip=True,
                                    fill_mode='nearest')
test_datagen=ImageDataGenerator(rescale=1./255)

In [ ]:
train = train_datagen.flow_from_directory(
        TRAIN_DIR,
        target_size=(224,224),
        batch_size=32,
        shuffle=True,
        class_mode='categorical')

test=test_datagen.flow_from_directory(TEST_DIR,
                                    batch_size = 32, target_size = (224,224), class_mode = 'categorical')

Found 776 images belonging to 8 classes.
Found 491 images belonging to 8 classes.


In [ ]:
mobile = MobileNetV2(input_shape=(224,224,3),weights="imagenet",include_top=False)

In [ ]:
mobile.trainable = False

In [ ]:
x=Flatten()(mobile.output)
pred = Dense(8,activation='softmax')(x)

In [ ]:
model=Model(inputs=mobile.input,outputs=pred)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_2[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                                           

In [ ]:
early_stop=EarlyStopping(monitor="val_loss",mode="min",verbose=1,patience=5)

In [ ]:
model.compile(loss="categorical_crossentropy",
              optimizer="adam",
              metrics=["accuracy"])

In [ ]:
model.fit(train,
          validation_data=test,
          epochs=20,
          callbacks=[early_stop])

Epoch 1/20
25/25 [==============================] - 34s 1s/step - loss: 8.2909 - accuracy: 0.4485 - val_loss: 5.8676 - val_accuracy: 0.6151
Epoch 2/20
25/25 [==============================] - 29s 1s/step - loss: 1.8591 - accuracy: 0.7874 - val_loss: 3.5031 - val_accuracy: 0.7149
Epoch 3/20
25/25 [==============================] - 29s 1s/step - loss: 1.0999 - accuracy: 0.8428 - val_loss: 3.3531 - val_accuracy: 0.7271
Epoch 4/20
25/25 [==============================] - 29s 1s/step - loss: 1.1848 - accuracy: 0.8582 - val_loss: 4.3499 - val_accuracy: 0.7149
Epoch 5/20
25/25 [==============================] - 29s 1s/step - loss: 0.9208 - accuracy: 0.8866 - val_loss: 3.8470 - val_accuracy: 0.7088
Epoch 6/20
25/25 [==============================] - 30s 1s/step - loss: 0.6194 - accuracy: 0.9304 - val_loss: 4.6336 - val_accuracy: 0.6965
Epoch 7/20
25/25 [==============================] - 29s 1s/step - loss: 0.5564 - accuracy: 0.9394 - val_loss: 3.5983 - val_accuracy: 0.7271
Epoch 8/20
25/25 [==

In [ ]:
model.save("/content/drive/MyDrive/kvasir_small/mobilenetv2_epoch20_lr0.001.hdf5")